In [ ]:
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0,MobileNet  
#from tensorflow.keras.applications import MobilNet
from tensorflow.keras.models import Model
import numpy as np
import random
import time
from numpy.linalg import norm
import PIL
from PIL import Image
import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.layers import GlobalAveragePooling2D, Concatenate

In [ ]:
# Load pre-trained SqueezeNet model without top (fully connected) layers
pure_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=True, input_shape = (224,224,3))
pure_model.summary()

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense
embedding_size = 1024

# Load pre-trained SqueezeNet model without top (fully connected) layers
base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape = (224,224,3))

#Remove the last layer of MobileNet
base_model_output = base_model.layers[-1].output

x = base_model.output

# Step 3: Adjust the architecture for retrieval
x = GlobalAveragePooling2D()(x)

# Add the final Dense layer with embedding_size
embeddings = Dense(embedding_size)(x)

# Add L2 normalization layer


class L2Normalization(tf.keras.layers.Layer):
    def __init__(self, axis=-1, epsilon=1e-12, **kwargs):
        super(L2Normalization, self).__init__(**kwargs)
        self.axis = axis
        self.epsilon = epsilon

    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=self.axis, epsilon=self.epsilon)
    
    def get_config(self):
        config = super(L2Normalization, self).get_config()
        config.update({'axis': self.axis, 'epsilon': self.epsilon})
        return config

    # Apply L2 normalization
    
embeddings = tf.keras.layers.Dense(embedding_size)(x)
embeddings = L2Normalization()(embeddings)


feature_extraction_model1 = Model(inputs=base_model.input, outputs=embeddings)

feature_extraction_model1.summary()

In [ ]:
# Load the base model
base_model2 = EfficientNetB0(weights='imagenet', include_top=False,input_shape = (224,224,3))
base_model2.summary()

 ormalization)                                                                                    
                                                                                                  
 block4c_drop (Dropout)      (None, 14, 14, 80)           0         ['block4c_project_bn[0][0]']  
                                                                                                  
 block4c_add (Add)           (None, 14, 14, 80)           0         ['block4c_drop[0][0]',        
                                                                     'block4b_add[0][0]']         
                                                                                                  
 block5a_expand_conv (Conv2  (None, 14, 14, 480)          38400     ['block4c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5a_e

 block5b_add (Add)           (None, 14, 14, 112)          0         ['block5b_drop[0][0]',        
                                                                     'block5a_project_bn[0][0]']  
                                                                                                  
 block5c_expand_conv (Conv2  (None, 14, 14, 672)          75264     ['block5b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5c_expand_bn (BatchNo  (None, 14, 14, 672)          2688      ['block5c_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block5c_expand_activation   (None, 14, 14, 672)          0         ['block5c_expand_bn[0][0]']   
 (Activati

 ormalization)                                                                                    
                                                                                                  
 block6b_expand_conv (Conv2  (None, 7, 7, 1152)           221184    ['block6a_project_bn[0][0]']  
 D)                                                                                               
                                                                                                  
 block6b_expand_bn (BatchNo  (None, 7, 7, 1152)           4608      ['block6b_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6b_expand_activation   (None, 7, 7, 1152)           0         ['block6b_expand_bn[0][0]']   
 (Activation)                                                                                     
          

 block6c_add (Add)           (None, 7, 7, 192)            0         ['block6c_drop[0][0]',        
                                                                     'block6b_add[0][0]']         
                                                                                                  
 block6d_expand_conv (Conv2  (None, 7, 7, 1152)           221184    ['block6c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block6d_expand_bn (BatchNo  (None, 7, 7, 1152)           4608      ['block6d_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6d_expand_activation   (None, 7, 7, 1152)           0         ['block6d_expand_bn[0][0]']   
 (Activati

                                                                                                  
 top_bn (BatchNormalization  (None, 7, 7, 1280)           5120      ['top_conv[0][0]']            
 )                                                                                                
                                                                                                  
 top_activation (Activation  (None, 7, 7, 1280)           0         ['top_bn[0][0]']              
 )                                                                                                
                                                                                                  
Total params: 4049571 (15.45 MB)
Trainable params: 4007548 (15.29 MB)
Non-trainable params: 42023 (164.16 KB)
__________________________________________________________________________________________________


In [ ]:
# Now define the layers for the second parallel network
# You can define them similar to the first one
layer4 = base_model2.get_layer('block2a_expand_activation').output
layer5 = base_model2.get_layer('block4a_expand_activation').output
layer6 = base_model2.get_layer('block6a_expand_activation').output
# Apply Global Average Pooling to each feature from the third layer
layer4 = GlobalAveragePooling2D()(layer4)

# Apply Global Average Pooling to each feature from the fourth layer
layer5 = GlobalAveragePooling2D()(layer5)


layer6 = GlobalAveragePooling2D()(layer6)

# Concatenate the feature vectors from the second set of layers
concatenated_features2 = Concatenate()([layer4, layer5, layer6])

# Create a model for feature extraction
feature_extraction_model2 = Model(inputs= base_model2.input, outputs= concatenated_features2)


feature_extraction_model2.summary()

 ormalization)                                                                                    
                                                                                                  
 block4c_drop (Dropout)      (None, 14, 14, 80)           0         ['block4c_project_bn[0][0]']  
                                                                                                  
 block4c_add (Add)           (None, 14, 14, 80)           0         ['block4c_drop[0][0]',        
                                                                     'block4b_add[0][0]']         
                                                                                                  
 block5a_expand_conv (Conv2  (None, 14, 14, 480)          38400     ['block4c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5a_e

 block5b_add (Add)           (None, 14, 14, 112)          0         ['block5b_drop[0][0]',        
                                                                     'block5a_project_bn[0][0]']  
                                                                                                  
 block5c_expand_conv (Conv2  (None, 14, 14, 672)          75264     ['block5b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5c_expand_bn (BatchNo  (None, 14, 14, 672)          2688      ['block5c_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block5c_expand_activation   (None, 14, 14, 672)          0         ['block5c_expand_bn[0][0]']   
 (Activati

In [ ]:
#Create list of files of dataset
extensions = extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.bmp', '.BMP']

def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

In [ ]:
#obtain file names 
import os
# path to the datasets
root_dir = 'C:\\Data Drive\\Datasets\\corel-1k\\dataset\\training_set'
filenames = sorted(get_file_list(root_dir))

In [ ]:
#assigning labels to dataset
import pathlib
data_dir = pathlib.Path(root_dir)
data_dir
dataset_labels=[]

object_categories_dict = {
    'beaches': list(data_dir.glob('beaches/*')),
    'bus': list(data_dir.glob('bus/*')),
    'dinosaurs': list(data_dir.glob('dinosaurs/*')),
    'elephants': list(data_dir.glob('elephants/*')),
    'flowers': list(data_dir.glob('flowers/*')),
    'foods': list(data_dir.glob('foods/*')),
    'horses': list(data_dir.glob('horses/*')),
    'monuments': list(data_dir.glob('monuments/*')),
    'mountains_and_snow': list(data_dir.glob('mountains_and_snow/*')),
    'peolpe_and_villages_in_Africa': list(data_dir.glob('peolpe_and_villages_in_Africa/*')),
}
object_labels_dict = {
    'beaches': 0, 
    'bus': 1, 
    'dinosaurs': 2,
    'elephants': 3, 
    'flowers': 4, 
    'foods': 5,
    'horses': 6,
    'monuments': 7, 
    'mountains_and_snow': 8, 
    'peolpe_and_villages_in_Africa': 9
}
#class_to_numeric = {class_label: i for i, class_label in enumerate(set(class_labels))}
for object_category,images in object_categories_dict.items():
    for image in images:
        dataset_labels.append(object_labels_dict[object_category])


In [ ]:
len(filenames)

In [ ]:
import cv2
X, y = [], []
for object_category,filenames in object_categories_dict.items():
    for filename in filenames:
        img = cv2.imread(str(filename))  # Convert Path object to string
        resized_img = cv2.resize(img, (224, 224))
        X.append(resized_img)
        y.append(object_labels_dict[object_category])

In [ ]:
X = np.array(X) 
y = np.array(y)

In [ ]:
len(y)

In [ ]:

x = X / 255.0  # Normalize pixel values to the range [0, 1]


In [ ]:
print("x dimensions:", x.shape)
print("y dimensions:", y.shape)

In [ ]:
import timeit


start_time = timeit.default_timer()
features_model1 = feature_extraction_model1.predict(x)
end_time = timeit.default_timer()
elapsed_time = end_time - start_time  # Calculate the elapsed time
print(f"Elaapsed Time: {elapsed_time:.4f} seconds")

In [ ]:
import timeit
start_time = timeit.default_timer()
features_model2 = feature_extraction_model2.predict(x)
end_time = timeit.default_timer()
elapsed_time = end_time - start_time  # Calculate the elapsed time
print(f"Elaapsed Time: {elapsed_time:.4f} seconds")

In [ ]:
# Concatenate both concatenated feature vectors
extracted_features = Concatenate()([features_model1,features_model2])


In [ ]:
# Assign labels to the extracted features
features_with_labels = list(zip(extracted_features, y))

In [ ]:


# Example usage: Iterating over features with their labels
for feature, label in features_with_labels:
    # Access the feature vector and its corresponding label
    print("Feature shape:", feature.shape)
    print("Label:", label)
    # Perform any operations using the feature and label

In [ ]:
# Reshape the extracted features array
features = tf.reshape(extracted_features, (len(extracted_features), -1))
print("Reshaped features shape:", features.shape)


In [ ]:
# Assuming features is your tensor
num_images, num_features_per_image = features.shape

# Printing the values
print("Number of images =", num_images)
print("Number of features per image =", num_features_per_image)

In [ ]:
 #Assuming 'features' is a NumPy array containing your extracted features
np.save('extracted_features_using_fusion_corel-1k.npy', features)

# Reading and extracting features of query images 

In [ ]:
#obtain file names 
import os
# path to the datasets
root_dir_query_images= 'C:\\Data Drive\\Datasets\\corel-1k\\dataset\\test_set'
filenames_query_images = sorted(get_file_list(root_dir_query_images))

In [ ]:
filenames_query_images[:5] 

In [ ]:
num_query_images = len(filenames_query_images)

num_query_images

In [ ]:
#assigning labels to dataset
import pathlib
data_dir_query = pathlib.Path(root_dir_query_images)
data_dir_query
dataset_labels_query=[]

object_categories_dict_q = {
    'beaches': list(data_dir_query.glob('beaches/*')),
    'bus': list(data_dir_query.glob('bus/*')),
    'dinosaurs': list(data_dir_query.glob('dinosaurs/*')),
    'elephants': list(data_dir_query.glob('elephants/*')),
    'flowers': list(data_dir_query.glob('flowers/*')),
    'foods': list(data_dir_query.glob('foods/*')),
    'horses': list(data_dir_query.glob('horses/*')),
    'monuments': list(data_dir_query.glob('monuments/*')),
    'mountains_and_snow': list(data_dir_query.glob('mountains_and_snow/*')),
    'peolpe_and_villages_in_Africa': list(data_dir_query.glob('peolpe_and_villages_in_Africa/*')),
}
object_labels_dict_q = {
    'beaches': 0, 
    'bus': 1, 
    'dinosaurs': 2,
    'elephants': 3, 
    'flowers': 4, 
    'foods': 5,
    'horses': 6,
    'monuments': 7, 
    'mountains_and_snow': 8, 
    'peolpe_and_villages_in_Africa': 9
}
#class_to_numeric = {class_label: i for i, class_label in enumerate(set(class_labels))}
for object_category,images in object_categories_dict_q.items():
    for image in images:
        dataset_labels_query.append(object_labels_dict_q[object_category])


In [ ]:
#resize query images 
X_q, y_q = [], []
for object_category,filenames_query_images in object_categories_dict_q.items():
    for filename in filenames_query_images:
        img = cv2.imread(str(filename))  # Convert Path object to string
        resized_img = cv2.resize(img, (224, 224))
        X_q.append(resized_img)
        y_q.append(object_labels_dict_q[object_category])

In [ ]:
X_q = np.array(X_q) 
y_q = np.array(y_q)

In [ ]:
x_q = X_q / 255.0  # Normalize pixel values to the range [0, 1]

In [ ]:
len(y_q)

In [ ]:
print("x_q dimensions:", x_q.shape)
print("y_q dimensions:", y_q.shape)

In [ ]:
y_q

In [ ]:
#extract features of query images 
features_query_model1 = feature_extraction_model1.predict(x_q)
features_query_model2 = feature_extraction_model2.predict(x_q)
# Concatenate both concatenated feature vectors
extracted_features_query_images = Concatenate()([features_query_model1,features_query_model2])



In [ ]:
# Assign labels to the extracted features
features_with_labels_q = list(zip(extracted_features_query_images, y_q))

In [ ]:
# Example usage: Iterating over features with their labels
for feature, label in features_with_labels_q:
    # Access the feature vector and its corresponding label
    print("Feature shape:", feature.shape)
    print("Label:", label)
    # Perform any operations using the feature and label

In [ ]:
# Reshape the extracted features array
features_query_images = tf.reshape(extracted_features_query_images, (len(extracted_features_query_images), -1))
print("Reshaped features shape:", features_query_images.shape)


In [ ]:
#Assuming features is your tensor
num_images_q, num_features_per_image_q = features_query_images.shape

# Printing the values
print("Number of images =", num_images_q)
print("Number of features per image =", num_features_per_image_q)

# Perform Retrieval

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

def ret_images_soi(query_features, extracted_features, radius=None, top_k=5, metric="euclidean"):
    """
    Perform image retrieval using Sphere of Influence (SOI) similarity measure.

    Args:
        query_features (ndarray): Query image feature vector (1, D)
        extracted_features (ndarray): Dataset of feature vectors (N, D)
        radius (float): SOI threshold; if None, dynamically computed
        top_k (int): Number of similar images to retrieve
        metric (str): Distance metric ('euclidean' or 'cosine')

    Returns:
        retrieved_images_with_labels (list): Retrieved images and their labels
        retrieved_labels (list): Labels of retrieved images
        top_indices (list): Indices of retrieved images
        retrieval_time (float): Time taken for retrieval
    """

    print("\nRetrieval using Sphere of Influence (SOI)\n")
    
    # Start timing
    start_time = time.time()


    query_features = np.array(query_features)  # Ensure NumPy array
    query_features = query_features.reshape(1, -1)  # Ensure 2D shape
    distances = cdist(extracted_features, query_features, metric=metric).flatten()



    # Compute pairwise distances between query and dataset features
    #distances = cdist(extracted_features, query_features.reshape(1, -1), metric=metric).flatten()


    # Compute SOI radius dynamically if not provided
    if radius is None:
        radius = np.mean(distances)  # Adaptive radius selection

    # Find indices within SOI
    within_soi = np.where(distances <= radius)[0]

    # Sort by distance and retain top-k results
    sorted_indices = within_soi[np.argsort(distances[within_soi])][:top_k]
    top_distances = distances[sorted_indices]

    end_time = time.time()
    retrieval_time = end_time - start_time

    print(f"SOI Radius: {radius:.4f}")
    print(f"Retrieval Time: {retrieval_time:.4f} seconds")
    print("Top Distances:", top_distances)

    # Plot query image
    plt.figure(figsize=(15, 2))
    plt.subplot(1, top_k + 1, 1)
    plt.imshow(x_q[query_index])
    plt.title("Query Image")

    # Plot retrieved images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, idx in enumerate(sorted_indices, start=2):
        plt.subplot(1, top_k + 1, i)
        plt.imshow(x[idx])
        plt.title(f"Image {i-1}\nDistance: {top_distances[i-2]:.4f}")
        
        # Store retrieved images and labels
        retrieved_images_with_labels.append((x[idx], y[idx]))
        retrieved_labels.append(y[idx])

    plt.show()

    return retrieved_images_with_labels, retrieved_labels, sorted_indices, retrieval_time


In [ ]:
def calculate_metrics(query_labels, retrieved_labels,k):
    if len(query_labels) != 1:
        raise ValueError("The query should have a single label.")

    query_label = query_labels[0]
    # Ensure retrieved_labels is a list
    if not isinstance(retrieved_labels, list):
        retrieved_labels = [retrieved_labels]
        
    # Ensure k does not exceed the length of retrieved_labels
    k = min(k, len(retrieved_labels))    

    #for k in k_values:
    top_k_retrieved = retrieved_labels[:k]
    
    # Calculate true positives
    true_positive = sum(1 for label in top_k_retrieved if label == query_label)
    
    # Calculate false positives
    false_positive = k - true_positive
    
    # Calculate false negatives
    false_negative = 1 if true_positive == 0 else 0  # There is a false negative if no true positive
    
    # Calculate true negatives
    true_negative = k - true_positive  # Assuming binary classification
    
    # Calculate accuracy at k
    accuracy_at_k = (true_positive + true_negative) / k
    
    # Calculate precision at k
    precision_at_k = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    
    # Calculate recall at k
    recall_at_k = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    
    # Calculate F1 score at k
    f_measure_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0.0
    
    
    return precision_at_k, recall_at_k, accuracy_at_k, f_measure_at_k



In [ ]:
import pandas as pd

def store_metrics_to_excel(metrics_data, retrieval_methods, filename):
    # Create a Pandas Excel writer using XlsxWriter as the engine
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    
    # Loop over the retrieval methods
    for method in retrieval_methods:
        # Convert the dictionary to a DataFrame
        df = pd.DataFrame(metrics_data[method])
        
        # Write each DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=method, index=False)
    
    # Close the Pandas Excel writer and save the file
    writer.save()

In [ ]:
#Retrieving top 20 images
k = 5

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'SOI': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    
    # Retrieve images using euclidean 
    retrieved_images_with_labels, retrieved_labels, top_indices,retrieval_time = ret_images_soi(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['soi'][f"Query {i + 1}"] = [precision, recall, f1, accuracy, retrieval_time]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['soi'])

In [ ]:
#Retrieving top 20 images
k = 25

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_retrieving_top25_images.xlsx')


In [ ]:
#Retrieving top 20 images
k = 20

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_retrieving_top20_images.xlsx')


In [ ]:
#Retrieving top 10 images
k = 10

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_retrieving_top10_images.xlsx')


In [ ]:
k=5

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(extracted_features_query_images[query_index], (1, -1))
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k=5)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_final.xlsx')


# End of Code

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib
import matplotlib.pyplot as plt
#Retrieve images using cosine similarity
def ret_images_cosine_similarity(query_features):
    similarity_scores = cosine_similarity(query_features, embedded_features)
    #print(similarity_scores)
    # Get the indices of the top 5 most similar images (excluding the query image itself)
    top_indices = np.argsort(similarity_scores[0])[::-1][1:6]
    #print(top_indices)
    # Plot the query image
    plt.figure(figsize=(10, 2))
    plt.subplot(1, 6, 1)
    plt.imshow(x[query_index])
    plt.title("Query Image")

    # Plot the top 5 similar images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, index in enumerate(top_indices, start=2):
        print("Index:", index)
        plt.subplot(1, 6, i)
        plt.imshow(x[index])
        plt.title(f"Similar Image {i-1}")
        # Append the retrieved image and its label to the list
        retrieved_images_with_labels.append((x[index], y[index]))
    
        retrieved_labels.append(y[index])
    plt.tight_layout()    
    plt.show()
    return retrieved_images_with_labels,retrieved_labels

In [ ]:
#retrieve images using cosine similarity
retrieved_images_with_labels,retrieved_labels = ret_images_cosine_similarity(query_features)

In [ ]:
retrieved_labels

In [ ]:
print(type(retrieved_labels))

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def ret_images_euclidean_distance(query_features):
    # Calculate Euclidean distances
    distances = euclidean_distances(query_features, embedded_features)
    # Get the indices of the top 5 most similar images
    top_indices = np.argsort(distances[0])[:5]
    #print(top_distances)
    # Plot the query image
    plt.figure(figsize=(10, 2))
    plt.subplot(1, 6, 1)
    plt.imshow(x[query_index])
    plt.title("Query Image")

    # Plot the top 5 similar images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, index in enumerate(top_indices, start=2):
        plt.subplot(1, 6, i)
        plt.imshow(x[index])
        plt.title(f"Similar Image {i-1}")
        # Append the retrieved image and its label to the list
        retrieved_images_with_labels.append((x[index], y[index]))
        retrieved_labels.append(y[index])
    plt.tight_layout()    
    plt.show()
    return retrieved_images_with_labels,retrieved_labels,distances


In [ ]:
retrieved_images_with_labels,retrieved_labels,distances = ret_images_euclidean_distance(query_features)

In [ ]:
retrieved_labels

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree

def ret_images_knn(query_features, k=5):
    # Fit a Nearest Neighbors model
    nn_model = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
    nn_model.fit(embedded_features)

    # Query for the nearest neighbors
    distances, top_indices = nn_model.kneighbors(query_features)
    print(distances) 
    # Plot the query image
    plt.figure(figsize=(15, 2))
    plt.subplot(1, 6, 1)
    plt.imshow(x[query_index])
    plt.title("Query Image")

    # Plot the top 5 similar images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, (index, distance) in enumerate(zip(top_indices[0], distances[0]), start=2):
        plt.subplot(1, 6, i)
        plt.imshow(x[index])
        plt.title(f"Similar Image {i-1}\nDistance: {distance:.4f}")
        # Append the retrieved image and its label to the list
        retrieved_images_with_labels.append((x[index], y[index]))
        retrieved_labels.append(y[index])
    #plt.tight_layout()    
    plt.show()
    return retrieved_images_with_labels,retrieved_labels,top_indices


    plt.show()


In [ ]:
# Perform retrieval using KNN
retrieved_images_with_labels,retrieved_labels,top_indices = ret_images_knn(query_features, k=5)

In [ ]:
retrieved_labels

In [ ]:
query_labels

In [ ]:
type(query_labels)

In [ ]:
type(retrieved_labels)

In [ ]:
def calculate_metrics(query_labels, retrieved_labels,k):
    if len(query_labels) != 1:
        raise ValueError("The query should have a single label.")

    query_label = query_labels[0]
    # Ensure retrieved_labels is a list
    if not isinstance(retrieved_labels, list):
        retrieved_labels = [retrieved_labels]
        
    # Ensure k does not exceed the length of retrieved_labels
    k = min(k, len(retrieved_labels))    

    #for k in k_values:
    top_k_retrieved = retrieved_labels[:k]
    
    # Calculate true positives
    true_positive = sum(1 for label in top_k_retrieved if label == query_label)
    
    # Calculate false positives
    false_positive = k - true_positive
    
    # Calculate false negatives
    false_negative = 1 if true_positive == 0 else 0  # There is a false negative if no true positive
    
    # Calculate true negatives
    true_negative = k - true_positive  # Assuming binary classification
    
    # Calculate accuracy at k
    accuracy_at_k = (true_positive + true_negative) / k
    
    # Calculate precision at k
    precision_at_k = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    
    # Calculate recall at k
    recall_at_k = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    
    # Calculate F1 score at k
    f_measure_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0.0
    
    
    return precision_at_k, recall_at_k, accuracy_at_k, f_measure_at_k



In [ ]:
k = 5
# Example usage
precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels,k)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)


In [ ]:
import numpy as np

def calculate_relevance_scores(distances, epsilon=1e-6):
    # Convert distances to relevance scores
    relevance_scores = 1 / (np.array(distances) + epsilon)
    return relevance_scores

In [ ]:
print(distances)
relevance_scores = calculate_relevance_scores(distances)
relevance_scores

In [ ]:
def calculate_ndcg(relevance_scores, k=None):
    # Sort relevance scores in descending order
    sorted_scores = np.sort(relevance_scores)[::-1]
    if k is None:
        k = len(relevance_scores)

    # Calculate DCG (Discounted Cumulative Gain)
    dcg = np.sum((2 ** sorted_scores - 1) / np.log2(np.arange(2, len(sorted_scores) + 2)))

    # Calculate IDCG (Ideal Discounted Cumulative Gain)
    num_rel = min(len(relevance_scores), k)
    ideal_scores = np.sort(1 / np.arange(1, num_rel + 1))[::-1]
    idcg = np.sum((2 ** ideal_scores - 1) / np.log2(np.arange(2, len(ideal_scores) + 2)))
    
    # Check if IDCG is zero to avoid division by zero
    if idcg == 0:
        return 0

    # Calculate NDCG (Normalized Discounted Cumulative Gain)
    ndcg = dcg / idcg
    return ndcg

In [ ]:
ndcg = calculate_ndcg(relevance_scores)
print("NDCG:", ndcg)